In [1]:
from collections import OrderedDict
from torchvision import datasets, transforms
import syft as sy # Syft 0.5.0
import numpy as np
import torch # Torch 1.8.1
import time
import tenseal as ts
from typing import Dict
import opacus

# Connect to Data Owner 1 

In [2]:
duet_1 = sy.duet("bfdb92b9d79738e803e4c55309e24d11")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 2cbbe8a7a91690e437adef830f822d7f

♫♫♫ > ...waiting for partner to connect...


C:\Users\davit\anaconda3\envs\NEW\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\davit\anaconda3\envs\NEW\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


# Connect to Data Owner 2

In [3]:
duet_2 = sy.duet("ad563827b39279fd2de26bc0834e6c9b")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: f18f4172e309823013f6db7b14fdbe96

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


# Connect to Data Owner 3

In [4]:
duet_3 = sy.duet("705d560f9db03eef1cb78caa5c8698cf")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 3f7952a3101e0a7fada920a80d511c8d

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


# Define Model

In [5]:
class classify(sy.Module):
    def __init__(self, torch_ref, num_classes=2):
        super(classify, self).__init__(torch_ref=torch_ref)
         
        self.conv1=self.torch_ref.nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        self.bn1=self.torch_ref.nn.BatchNorm2d(num_features=12, momentum=0)
        self.relu1=self.torch_ref.nn.ReLU()        
        self.pool=self.torch_ref.nn.MaxPool2d(kernel_size=2)
        self.conv2=self.torch_ref.nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=self.torch_ref.nn.ReLU()
        self.conv3=self.torch_ref.nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=self.torch_ref.nn.BatchNorm2d(num_features=32, momentum=0)
        self.relu3=self.torch_ref.nn.ReLU()
        self.fc=self.torch_ref.nn.Linear(in_features=32 * 112 * 112,out_features=num_classes)

    def forward(self, input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)            
        output=output.view(-1,32*112*112)
        output=self.fc(output)
            
        return output

In [6]:
local_model = classify(torch)

# Send Models to Data Owners

In [7]:
# Send to the domain owner 1
remote_model_1 = local_model.send(duet_1)
remote_model_1.train()
remote_torch_1 = duet_1.torch

In [8]:
params_1 = remote_model_1.parameters()
optimizer_1 = remote_torch_1.optim.Adam(params=params_1, lr=0.2)

In [9]:
# Send to the domain owner 2
remote_model_2 = local_model.send(duet_2)
remote_torch_2 = duet_2.torch

In [10]:
params_2 = remote_model_2.parameters()
optimizer_2 = remote_torch_2.optim.Adam(params=params_2, lr=0.2)

In [11]:
# Send to the domain owner 3
remote_model_3 = local_model.send(duet_3)
remote_torch_3 = duet_3.torch

In [12]:
params_3 = remote_model_3.parameters()
optimizer_3 = remote_torch_3.optim.Adam(params=params_3, lr=0.2)

# Define Training Function and Training Variables

In [13]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):
    losses = []
    
    for i in range(iterations):
        
        optim.zero_grad()
        
        output = model(data_ptr)
        loss = torch_ref.nn.functional.cross_entropy(output, target_ptr.long())
        loss.retain_grad()

        loss_value = loss.get(
            reason="To evaluate training progress", request_block=True, timeout_secs=60, delete_obj=False
        )
        
        print("Epoch", i, "loss", loss_value)
            
        losses.append(loss_value)
        
        loss.backward()
        
        optim.step()
          
    return losses

# Get Data Owner 1 Data Pointer and Label Pointer

In [14]:
data_ptr_1 = duet_1.store[0]
label_ptr_1 = duet_1.store[1]

# Get Data Owner 2 Data Pointer and Label Pointer

In [15]:
data_ptr_2 = duet_2.store[0]
label_ptr_2 = duet_2.store[1]

# Get Data Owner 3 Data Pointer and Label Pointer

In [16]:
data_ptr_3 = duet_3.store[0]
label_ptr_3 = duet_3.store[1]

# Train Data Owner 1

In [17]:
iterations = 100
start = time.time()
losses = train(iterations, remote_model_1, remote_torch_1, optimizer_1, data_ptr_1, label_ptr_1)
print('It took', time.time()-start, 'seconds.')

Epoch 0 loss tensor(0.7901, requires_grad=True)
Epoch 1 loss tensor(23565.4395, requires_grad=True)
Epoch 2 loss tensor(4919.6392, requires_grad=True)
Epoch 3 loss tensor(6221.1699, requires_grad=True)
Epoch 4 loss tensor(4061.5581, requires_grad=True)
Epoch 5 loss tensor(1458.7731, requires_grad=True)
Epoch 6 loss tensor(357.0091, requires_grad=True)
Epoch 7 loss tensor(613.0756, requires_grad=True)
Epoch 8 loss tensor(500.4550, requires_grad=True)
Epoch 9 loss tensor(311.9389, requires_grad=True)
Epoch 10 loss tensor(164.3881, requires_grad=True)
Epoch 11 loss tensor(62.8827, requires_grad=True)
Epoch 12 loss tensor(7.9855, requires_grad=True)
Epoch 13 loss tensor(46.2871, requires_grad=True)
Epoch 14 loss tensor(45.6435, requires_grad=True)
Epoch 15 loss tensor(22.4886, requires_grad=True)
Epoch 16 loss tensor(13.3550, requires_grad=True)
Epoch 17 loss tensor(19.0922, requires_grad=True)
Epoch 18 loss tensor(23.2145, requires_grad=True)
Epoch 19 loss tensor(25.4693, requires_grad=Tr

# Train Data Owner 2

In [18]:
iterations = 100
start = time.time()
losses = train(iterations, remote_model_2, remote_torch_2, optimizer_2, data_ptr_2, label_ptr_2)
print('It took', time.time()-start, 'seconds.')

Epoch 0 loss tensor(0.7936, requires_grad=True)
Epoch 1 loss tensor(21373.2070, requires_grad=True)
Epoch 2 loss tensor(4606.0483, requires_grad=True)
Epoch 3 loss tensor(4739.9736, requires_grad=True)
Epoch 4 loss tensor(3339.1079, requires_grad=True)
Epoch 5 loss tensor(1069.8982, requires_grad=True)
Epoch 6 loss tensor(171.9167, requires_grad=True)
Epoch 7 loss tensor(172.1994, requires_grad=True)
Epoch 8 loss tensor(156.0282, requires_grad=True)
Epoch 9 loss tensor(54.3112, requires_grad=True)
Epoch 10 loss tensor(24.1601, requires_grad=True)
Epoch 11 loss tensor(3.6859, requires_grad=True)
Epoch 12 loss tensor(34.8413, requires_grad=True)
Epoch 13 loss tensor(12.3622, requires_grad=True)
Epoch 14 loss tensor(38.4578, requires_grad=True)
Epoch 15 loss tensor(47.0362, requires_grad=True)
Epoch 16 loss tensor(43.6774, requires_grad=True)
Epoch 17 loss tensor(38.4485, requires_grad=True)
Epoch 18 loss tensor(88.0137, requires_grad=True)
Epoch 19 loss tensor(61.1761, requires_grad=True

# Train Data Owner 3

In [19]:
iterations = 100
start = time.time()
losses = train(iterations, remote_model_3, remote_torch_3, optimizer_3, data_ptr_3, label_ptr_3)
print('It took', time.time()-start, 'seconds.')

Epoch 0 loss tensor(0.8109, requires_grad=True)
Epoch 1 loss tensor(21367.7871, requires_grad=True)
Epoch 2 loss tensor(5892.7070, requires_grad=True)
Epoch 3 loss tensor(2540.3552, requires_grad=True)
Epoch 4 loss tensor(2467.9924, requires_grad=True)
Epoch 5 loss tensor(263.6607, requires_grad=True)
Epoch 6 loss tensor(320.4272, requires_grad=True)
Epoch 7 loss tensor(404.5284, requires_grad=True)
Epoch 8 loss tensor(344.6219, requires_grad=True)
Epoch 9 loss tensor(237.4182, requires_grad=True)
Epoch 10 loss tensor(115.8260, requires_grad=True)
Epoch 11 loss tensor(24.4386, requires_grad=True)
Epoch 12 loss tensor(9.5104, requires_grad=True)
Epoch 13 loss tensor(5.9565, requires_grad=True)
Epoch 14 loss tensor(35.0970, requires_grad=True)
Epoch 15 loss tensor(46.1302, requires_grad=True)
Epoch 16 loss tensor(23.5955, requires_grad=True)
Epoch 17 loss tensor(8.3163, requires_grad=True)
Epoch 18 loss tensor(10.3554, requires_grad=True)
Epoch 19 loss tensor(13.7634, requires_grad=True)

# Get Updates

In [20]:
remote_model_1_updates = remote_model_1.get(
    request_block=True
).state_dict()

In [21]:
remote_model_1_updates

OrderedDict([('conv1.weight',
              tensor([[[[-1.3486, -1.3437, -1.9394],
                        [-1.2536, -1.1757, -1.5482],
                        [-0.8120, -1.1406, -1.1071]],
              
                       [[-1.3561, -1.5246, -1.6725],
                        [-1.1390, -1.2338, -1.8428],
                        [-0.5872, -1.0722, -1.3489]],
              
                       [[-1.1927, -1.3567, -1.9442],
                        [-1.2869, -1.4149, -1.7548],
                        [-1.1513, -1.0102, -1.1319]]],
              
              
                      [[[-1.0901, -0.5414,  2.1835],
                        [ 0.1013,  0.4123,  2.3458],
                        [ 0.2108,  0.5365,  2.0419]],
              
                       [[-1.2315, -0.6081,  1.9599],
                        [-0.0711,  0.2793,  2.1101],
                        [ 0.0682, -0.0783,  1.7752]],
              
                       [[-1.1769, -1.0139,  1.4225],
                        [-

In [22]:
remote_model_1_updates.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'bn3.weight', 'bn3.bias', 'bn3.running_mean', 'bn3.running_var', 'bn3.num_batches_tracked', 'fc.weight', 'fc.bias'])

In [23]:
remote_model_2_updates = remote_model_2.get(
    request_block=True
).state_dict()

In [24]:
remote_model_3_updates = remote_model_3.get(
    request_block=True
).state_dict()

# Average Updates

In [25]:
avg_updates = OrderedDict()
for key in remote_model_1_updates.keys():
    avg_updates[key] = (
        (remote_model_1_updates[key] +
        remote_model_2_updates[key] +
        remote_model_3_updates[key]) / 3
    )

# Load Average Weights

In [26]:
aggregated_model = classify(torch)

In [27]:
aggregated_model.load_state_dict(avg_updates)

In [28]:
#aggregated_model.eval()

In [29]:
#del avg_updates, remote_model_1_updates, remote_model_2_updates, remote_model_3_updates

# Test the Agregated Model

In [49]:
PIL_to_Tensor = transforms.Compose([
                transforms.Resize(size = (224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

In [50]:
test_images = datasets.ImageFolder('chest_xray/test', transform= PIL_to_Tensor)

In [52]:
test_data = []
test_labels = []

In [53]:
for index in range(0, 300):
    data_image = test_images[index][0].type(torch.long).numpy()
    test_data.append(data_image)
    test_labels.append(test_images[index][1])

In [54]:
test_data = torch.FloatTensor(test_data)
test_labels = torch.FloatTensor(test_labels)

In [55]:
test_data = test_data.tag("#Aggragted_Model_X")
test_labels = test_labels.tag("#Aggragted_Model_X")

In [56]:
preds = []
count = 0
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i].view(1, 3, 224, 224)
        y_hat = aggregated_model(sample)
        prediction = float(y_hat.argmax(dim=1).item())
        label = test_labels[i].item()
        if prediction == label:
            count += 1
            
print(f"Accuracy: {count/len(test_data)}")

Accuracy: 0.77
